# Toilets Berlin

### Import Packages

In [1]:
import pandas as pd
import sqlalchemy
import os
from dotenv import load_dotenv, find_dotenv
from functools import wraps
import datetime as dt

### Load variables from .env file

In [2]:
# load env data from .env file.
load_dotenv(find_dotenv(filename='.env'))

True

### Logging Wrapper

In [3]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Get Toilets from CSV

In [4]:
init_toilets = pd.read_excel("../data/csv/berlin-toillets.xlsx")
init_toilets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   LavatoryID               353 non-null    object 
 1   Description              353 non-null    object 
 2   City                     353 non-null    object 
 3   Street                   353 non-null    object 
 4   Number                   0 non-null      float64
 5   PostalCode               353 non-null    int64  
 6   Country                  353 non-null    object 
 7   Longitude                353 non-null    float64
 8   Latitude                 353 non-null    float64
 9   isOwnedByWall            353 non-null    int64  
 10  isHandicappedAccessible  353 non-null    int64  
 11  Price                    353 non-null    float64
 12  canBePayedWithCoins      353 non-null    int64  
 13  canBePayedInApp          353 non-null    int64  
 14  canBePayedWithNFC        3

## Data Cleaning Pipeline

### Init Pipeline

In [5]:
@log_step
def init_pipeline(df):
    return  df.copy()

### Rename Columns

In [6]:
@log_step
def rename_columns(df):
    df.columns = [str(x).lower() for x in df.columns]
    return df

### Drop Columns

In [7]:
@log_step
def drop_columns(df):
    return  df.drop(columns=["number"])

### Add Columns

In [8]:
@log_step
def add_columns(df):
    return  (
        df
        .assign(municipality_country = lambda x: x["city"] + ",DE")
        .assign(created_at = dt.datetime.now())
    )

### Drop Duplicates

In [9]:
@log_step
def drop_duplicates(df):
    return  df.drop_duplicates()

### Adjust Datatypes

In [ ]:

def adjust_datatypes(df):
    df["lavatoryid"] = df["lavatoryid"].astype("string")
    df["description"] = df["description"].astype("string")
    df["city"] = df["city"].astype("string")
    df["street"] = df["street"].astype("string")
    df["postalcode"] =df["postalcode"].astype("string")
    df["isownedbywall"] = df ["isownedbywall"].astype("boolean")
    df["ishandicappedaccessible"] = df["ishandicappedaccessible"].astype("boolean")
    df["price"] = df["price"].astype("int16")
    df["canbepayedwithcoins"] = df["canbepayedwithcoins"].astype("boolean")
    df["canbepayedinapp"] = df["canbepayedinapp"].astype("boolean")
    df["canbepayedwithnfc"] = df["canbepayedwithnfc"].astype("boolean")
    df["haschangingtable"] = df["haschangingtable"].astype("boolean")
    df["labelid"] = df["labelid"].astype("int32")
    df["hasurinal"] = df["hasurinal"].astype("boolean")
    df["fid"] = df["fid"].astype("string")
    
    
    return df


### Send to DB

In [ ]:
def send_to_DB(df, table_name, if_exists="replace"):      
    con = f'mysql+pymysql://{os.environ["DB_USER"]}:{os.environ["DB_PASSWORD"]}@{os.environ["DB_HOST"]}:{os.environ["DB_PORT"]}/{os.environ["DB_SCHEMA"]}'
    df.to_sql(
        table_name, 
        con=con, 
        if_exists=if_exists,
        index=False,
        dtype={
            'lavatoryid': sqlalchemy.types.VARCHAR(length=20),
            'city': sqlalchemy.types.VARCHAR(length=50),
            'description': sqlalchemy.types.VARCHAR(length=500),
            'street': sqlalchemy.types.VARCHAR(length=100),
            'country': sqlalchemy.types.VARCHAR(length=30),
            'fid': sqlalchemy.types.VARCHAR(length=30),
            "latitude" : sqlalchemy.types.Float(precision=8, asdecimal=True),
            "longitude" : sqlalchemy.types.Float(precision=8, asdecimal=True),
            'postalcode': sqlalchemy.types.VARCHAR(length=7),
            'municipality_country': sqlalchemy.types.VARCHAR(length=100),
            'created_at': sqlalchemy.types.DateTime(),
        }
    )
    engine = sqlalchemy.create_engine(con)
    with engine.connect() as engine:
        engine.execute('ALTER TABLE `berlin_toilets` ADD PRIMARY KEY (`lavatoryid`);')
        engine.execute('ALTER TABLE `berlin_toilets` ADD FOREIGN KEY (municipality_country) REFERENCES cities(municipality_country);')
    return df


## RUN Pipeline

In [ ]:
berlin_toilets = (
    init_toilets
        .pipe(init_pipeline)
        .pipe(rename_columns)
        .pipe(drop_columns)
        .pipe(add_columns) # Population, Drop Citys with no Population data
        .pipe(drop_duplicates)
        .pipe(adjust_datatypes)
        # Send to DB in cell below
)
berlin_toilets.info()

init_pipeline:
 shape=(353, 19) took 0:00:00.000171s

rename_columns:
 shape=(353, 19) took 0:00:00.000292s

drop_columns:
 shape=(353, 18) took 0:00:00.000716s

add_columns:
 shape=(353, 20) took 0:00:00.001659s

drop_duplicates:
 shape=(353, 20) took 0:00:00.008480s

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 0 to 352
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   lavatoryid               353 non-null    string        
 1   description              353 non-null    string        
 2   city                     353 non-null    string        
 3   street                   353 non-null    string        
 4   postalcode               353 non-null    string        
 5   country                  353 non-null    object        
 6   longitude                353 non-null    float64       
 7   latitude                 353 non-null    float64       
 8   isownedbyw

In [ ]:
send_to_DB(
       df=berlin_toilets, 
       table_name="berlin_toilets", 
       if_exists="replace"
)

,lavatoryid,description,city,street,postalcode,country,longitude,latitude,isownedbywall,ishandicappedaccessible,price,canbepayedwithcoins,canbepayedinapp,canbepayedwithnfc,haschangingtable,labelid,hasurinal,fid,municipality_country,created_at
0,Wall_101003,Toilette Ottmachauer Steig (oberhalb Badestell...,Berlin,"Krumme Lanke, Quermatenweg (0-24 Uhr)",14109,Deutschland,13.240575,52.453581,True,False,0,False,False,False,False,5,False,269.0,"Berlin,DE",2022-04-07 19:07:09.661086
1,Wall_112911,Toilette Uferweg,Berlin,"Schlachtensee , Am Schlachtensee ggü. 145 (0-...",14129,Deutschland,13.199229,52.436507,True,False,0,False,False,False,False,5,False,<NA>,"Berlin,DE",2022-04-07 19:07:09.661086
2,Wall_115753,Toilette Am Kiesteich 50,Berlin,Spektepark (0-24 Uhr),13589,Deutschland,13.164859,52.548840,True,False,0,False,False,False,False,5,False,<NA>,"Berlin,DE",2022-04-07 19:07:09.661086
3,Wall_116738,"Wall CT, Hubertusdamm",Berlin,Hubertusdamm ggü. 7 (0-24 Uhr),13125,Deutschland,13.471198,52.615671,True,True,0,True,True,True,True,1,True,868.0,"Berlin,DE",2022-04-07 19:07:09.661086
4,Wall_116739,"Wall CT, Lindenufer",Berlin,Lindenufer ggü. 10 (0-24 Uhr),13597,Deutschland,13.207764,52.537351,True,True,0,True,True,True,True,1,False,738.0,"Berlin,DE",2022-04-07 19:07:09.661086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Fremd_5,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,12049,Deutschland,13.417550,52.476810,False,True,0,False,False,False,False,6,False,<NA>,"Berlin,DE",2022-04-07 19:07:09.661086
349,Fremd_6,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,12049,Deutschland,13.417660,52.470190,False,True,0,False,False,False,False,6,False,<NA>,"Berlin,DE",2022-04-07 19:07:09.661086
350,Fremd_7,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,12101,Deutschland,13.386770,52.471300,False,True,0,False,False,False,False,6,False,<NA>,"Berlin,DE",2022-04-07 19:07:09.661086
351,Fremd_8,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,10965,Deutschland,13.409950,52.479400,False,True,0,False,False,False,False,6,False,<NA>,"Berlin,DE",2022-04-07 19:07:09.661086


In [ ]:
berlin_toilets.loc[berlin_toilets["price"] != 0]

,lavatoryid,description,city,street,postalcode,country,longitude,latitude,isownedbywall,ishandicappedaccessible,price,canbepayedwithcoins,canbepayedinapp,canbepayedwithnfc,haschangingtable,labelid,hasurinal,fid,municipality_country,created_at
